# Create Corpora

(C) 2023 by [Damir Cavar](http://damir.cavar.me/)

This notebook generates the datasets for the Ellipsis experiments with Large Language Models (LLMs) and other NLP tools.

This code is part of the [NLP-Lab](http://nlp-lab.org/) [Ellipsis Project](https://nlp-lab.org/ellipsis/).

In [142]:
import os
import regex as re
import pandas as pd
from ipydatagrid import DataGrid
import langcodes
import csv
from nltk.tokenize.treebank import TreebankWordDetokenizer
import random

In [125]:
re_entry = re.compile(r"\n+(?P<ellipsis>.+)\n^----\n(?P<fullform>^.+)\n(?P<rest>(^([AB]:\s+.+|#.+)\n)*)", re.MULTILINE |re.UNICODE)

In [126]:
def parse_txt(filename: str):
    ellipsis = []
    fullforms = []
    with open(filename, mode='r', encoding='utf-8') as f:
        text = f.read()
        for line in f:
            line = line.strip()
        counter = 0
        for match in re_entry.finditer(text):
            # print(match.group('ellipsis'))
            ellipsis.append(match.group('ellipsis').strip())
            # print(match.group('fullform'))
            fullforms.append(match.group('fullform').strip())
            #print(match.group('rest'))
            counter += 1
    return counter, ellipsis, fullforms

In [3]:
def parse_txt_format(filename: str, debug: bool=False) -> list:
    res = []
    if "ellipsis" in filename:
        with open(filename, mode='r', encoding='utf-8') as ifp:
            text = ifp.read()
        if debug:
            print(text)
        counter = 0
        for match in re_entry.finditer(text):
            #s = match.start()
            #e = match.end()
            counter += 1
        return counter
    return 0

In [127]:
directory = "corpus"

langdirs = [ x for x in os.listdir(directory) if len(x) == 3 ]
res = []
res_data = set()
for lfolder in langdirs:
    lang = langcodes.Language.get(lfolder).display_name()
    if lfolder == "chn":
        lang = "Mandarin Chinese"
    if lang.startswith("Unknown"): lang = lfolder
    for subdir, dirs, files in os.walk(os.path.join(directory, lfolder)):
        for file in files:
            if file.endswith(".txt"):
                full_path = os.path.join(subdir, file)
                DEBUG=False
                count, ellipsis, fullform = parse_txt(full_path) # , debug=DEBUG)
                if count > 0:
                    res.append( (lang, os.path.splitext(os.path.basename(full_path))[0], count) )
                if ellipsis:
                    res_data.update( set( [ (lang, ) + e for e in zip(ellipsis, fullform) ]) )
            elif file.endswith(".xml"):
                pass
res.sort(key=lambda x: x[2], reverse=True)
df = pd.DataFrame(res, columns=("lang", "type", "count"), index=None)
bylang = {}
for r in res:
    bylang[r[0]] = bylang.get(r[0], 0) + r[2]
bylang = list(bylang.items())
bylang.sort(key=lambda x: x[1], reverse=True)
df_bylang = pd.DataFrame(bylang, columns=("lang", "count"), index=None)

In [128]:
eng_pairs = [ (TreebankWordDetokenizer().detokenize(x[1].split()), TreebankWordDetokenizer().detokenize(x[2].split())) for x in res_data if x[0] == 'English' ]
#for e in eng_pairs:
#    print(e)
eng_pairs_labeled = [ (TreebankWordDetokenizer().detokenize(e[0].split()), 1) for e in eng_pairs ]
eng_pairs_labeled.extend( [ (TreebankWordDetokenizer().detokenize(e[1].split()), 0) for e in eng_pairs ] )

In [129]:
just_distractors = []
with open("treebank_sentences_no_ellipsis.txt", mode='r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line.startswith("#"):
            if line.startswith("# STOP"):
                break
            continue
        eng_pairs_labeled.append( (line, 0) )

In [112]:
random.shuffle(eng_pairs_labeled)

In [130]:
for x in eng_pairs_labeled[:10]:
    print(x)
count_ellipsis = sum( [ 1 for x in eng_pairs_labeled if x[1] == 1 ] )
count_non_ellipsis = sum( [ 1 for x in eng_pairs_labeled if x[1] == 0 ] )
print(f"Examples with ellipsis {count_ellipsis}\nExamples without ellipsis {count_non_ellipsis}")

('If I need to ask for money up front later, I will ___.', 1)
('Big cyclical companies are using "all the tricks they can ___ to stabilize earnings," says Mr. Sloan.', 1)
("Jill likes your story even though she hates Bill's ___.", 1)
("Many are quick to emphasize that just because the market can fall as fast as it did ___ Friday doesn't mean it will tank again, despite some disquieting similarities between now and October 1987.", 1)
('His wife also works for the paper, as did his father ___.', 1)
('Did Frank get married first, or ___ Larry ___?', 1)
("They didn't ___.", 1)
('With her keen on him, and him ___ on her, the party should be fun.', 1)
('"A lot of people think I will give away the store, but I can assure you I will not ___," he says.', 1)
('Why, she tried to learn you your book, she tried to learn you your manners, she tried to be good to you every way she knowed how ___.', 1)
Examples with ellipsis 559
Examples without ellipsis 646


In [131]:
def clean_string(s):
    res = re.sub(r"\s\s+", " ", s.replace("___", ""))
    return TreebankWordDetokenizer().detokenize(res.split())

In [132]:
clean_eng_pairs_labeled = [ (clean_string(x[0]), x[1]) for x in eng_pairs_labeled ]
first_task_data = pd.DataFrame(clean_eng_pairs_labeled, columns=["sentence", "ellipsis"])
print(first_task_data)

                                               sentence  ellipsis
0     If I need to ask for money up front later, I w...         1
1     Big cyclical companies are using "all the tric...         1
2     Jill likes your story even though she hates Bi...         1
3     Many are quick to emphasize that just because ...         1
4     His wife also works for the paper, as did his ...         1
...                                                 ...       ...
1200  Exports in October stood at $5.29 billion, a m...         0
1201  South Korea's economic boom, which began in 19...         0
1202  Government officials said exports at the end o...         0
1203  Despite the gloomy forecast, South Korea has r...         0
1204  From January to October, the nation's accumula...         0

[1205 rows x 2 columns]


In [133]:
for i in range(10):
    first_task_data = first_task_data.sample(frac=1).reset_index(drop=True)
    first_task_data.to_csv(f'task1_random_{i+1}.csv', header=None, index=None)

In [134]:
second_task_data = pd.DataFrame([ (clean_string(x[0]), x[0]) for x in eng_pairs ], columns=["without", "with"])
# print(second_task_data[:3])
for i in range(10):
    second_task_data = second_task_data.sample(frac=1).reset_index(drop=True)
    second_task_data.to_csv(f'task2_random_{i+1}.csv', header=None, index=None)

In [135]:
third_task_data = pd.DataFrame([ (clean_string(x[0]), x[1]) for x in eng_pairs ], columns=["without", "with"])
for i in range(10):
    third_task_data = third_task_data.sample(frac=1).reset_index(drop=True)
    third_task_data.to_csv(f'task3_random_{i+1}.csv', header=None, index=None)

# Convert ELLie to IEL Format



In [139]:
def read_ellie(fname):
    data = []
    with open(fname, newline='') as f:
        reader = csv.reader(f)
        next(reader, None) # skip header
        columns = ["ID", "Condition", "Sentence", "Construction", "Semantic_Role"]
        data = list(reader)
    return [ (x[2], 1) for x in data ]

In [140]:
ellie_file = os.path.join(".", "corpus", "eng", "ellipsis", "ELLie.csv")
ellie_data = read_ellie(ellie_file)
print(ellie_data[:2])

[('The photographer used the camera, and the reporter did too.', 1), ('The photographer used the camera, and the butcher did too.', 1)]


In [141]:
ellie_task_data = pd.DataFrame([ (clean_string(x[0]), x[1]) for x in ellie_data ], columns=["sentence", "ellipsis"])
for i in range(10):
    ellie_task_data = ellie_task_data.sample(frac=1).reset_index(drop=True)
    ellie_task_data.to_csv(f'task1_ellie_{i+1}.csv', header=None, index=None)